In [1]:
import sys

sys.path.append("..")

In [2]:
from transformers import (
    DataCollatorWithPadding,
    RobertaForSequenceClassification,
    RobertaTokenizerFast,
    TrainingArguments,
)

from adna.pylib import consts
from adna.pylib.adna_dataset import ADnaDataset
from adna.pylib.weighted_trainer import WeightedTrainer

## The model to fine tune

In [3]:
MODEL_PATH = consts.MT_DIR / "train" / "checkpoint-47888"

## Data augmentation parameters

In [4]:
REV_COMP_RATE = 0.5
TO_N_RATE = 0.02

## Training parameters

In [5]:
TRAIN_EPOCHS = 50
LEARNING_RATE = 1e-5
TRAIN_BATCH_SIZE = 96
EVAL_BATCH_SIZE = 96

MODEL_DIR = 'finetune'  # Save check points to this sub-directory

## Get the tokenizer

In [6]:
tokenizer_path = str(consts.MT_DIR)
tokenizer = RobertaTokenizerFast.from_pretrained(tokenizer_path)

## Get the datasets

In [7]:
train_dataset = ADnaDataset(
    "train",
    tokenizer,
    rev_comp_rate=REV_COMP_RATE,
    to_n_rate=TO_N_RATE,
    # limit=TRAIN_BATCH_SIZE,
)
eval_dataset = ADnaDataset("val", tokenizer)  # , limit=EVAL_BATCH_SIZE)

In [8]:
train_dataset[0]

{'input_ids': [0, 43, 484, 1031, 3813, 50, 39, 302, 307, 50, 304, 3503, 357, 2976, 373, 1019, 298, 37, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'label': tensor(0)}

## Get the model

In [9]:
model = RobertaForSequenceClassification.from_pretrained(
    MODEL_PATH, local_files_only=True
)

## Build the data collator

In [10]:
data_collator = DataCollatorWithPadding(
    tokenizer=tokenizer,
    padding="max_length",
    max_length=consts.MAX_LENGTH,
)

## Build the trainer

In [11]:
training_args = TrainingArguments(
    output_dir=consts.MT_DIR / MODEL_DIR,
    overwrite_output_dir=True,
    num_train_epochs=TRAIN_EPOCHS,
    learning_rate=LEARNING_RATE,
    per_device_train_batch_size=TRAIN_BATCH_SIZE,
    per_device_eval_batch_size=EVAL_BATCH_SIZE,
    evaluation_strategy="epoch",
    logging_strategy="epoch",
    save_strategy="epoch",
    seed=23,
)

In [12]:
trainer = WeightedTrainer(
    train_dataset,
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
)

## Train

In [13]:
trainer.train()

/museum/rafe/other/adna/.venv/lib/python3.10/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 287233
  Num Epochs = 50
  Instantaneous batch size per device = 96
  Total train batch size (w. parallel, distributed & accumulation) = 96
  Gradient Accumulation steps = 1
  Total optimization steps = 149650


Epoch,Training Loss,Validation Loss
1,0.178100,0.160432
2,0.174800,0.143676
3,0.172400,0.150264
4,0.167700,0.156057
5,0.165300,0.157614
6,0.163000,0.146097
7,0.160400,0.150507
8,0.158100,0.160236
9,0.154900,0.162002
10,0.152500,0.173886


***** Running Evaluation *****
  Num examples = 95744
  Batch size = 96
Saving model checkpoint to ../data/UF46992/finetune/checkpoint-2993
Configuration saved in ../data/UF46992/finetune/checkpoint-2993/config.json
Model weights saved in ../data/UF46992/finetune/checkpoint-2993/pytorch_model.bin
tokenizer config file saved in ../data/UF46992/finetune/checkpoint-2993/tokenizer_config.json
Special tokens file saved in ../data/UF46992/finetune/checkpoint-2993/special_tokens_map.json
***** Running Evaluation *****
  Num examples = 95744
  Batch size = 96
Saving model checkpoint to ../data/UF46992/finetune/checkpoint-5986
Configuration saved in ../data/UF46992/finetune/checkpoint-5986/config.json
Model weights saved in ../data/UF46992/finetune/checkpoint-5986/pytorch_model.bin
tokenizer config file saved in ../data/UF46992/finetune/checkpoint-5986/tokenizer_config.json
Special tokens file saved in ../data/UF46992/finetune/checkpoint-5986/special_tokens_map.json
***** Running Evaluation ***

KeyboardInterrupt: 

### It looks like epoch 2 is best

Path = ../data/UF46992/models/checkpoint-5986
